# Improving Image Matting using Comprehensive Samepling Sets

>Shahrian E, Rajan D, Price B, et al. Improving Image Matting Using Comprehensive Sampling Sets[C]// Computer Vision and Pattern Recognition. IEEE, 2013:636-643.

这篇文章本来是不想看的，但是<a href='https://github.com/CCODING04/DeepImageMatting/blob/master/Temporally%20Coherent%20and%20Spatially%20Accurate%20Video%20Matting.ipynb'>上文</a>提到了sampling-based methods，然后作者又说我关于如何提取图像中前景、背景的所有颜色的方法都源于这篇文章。  
还能说啥，看看呗Orz

## 1. 先看摘要
  
为什么先看摘要呢，当然因为它短小精悍啊= =  

文章解决了两个问题:  
1. 对图像像素采集的时候往往会遇到前景-背景色采集不到的限制
2. 前景-背景区域的颜色分布存在重叠现象
  
第一个问题的解决办法是通过增加采样像素的个数，以及尽量在边缘周围采样（即在边缘一定距离的位置采样）。这个解决办法很好理解，图像很大需要采样再做聚类等操作，这时候少数量肯定不行，容易缺失颜色，但是其实我们通过数学验证下就能发现其实一定数量的像素采集就能基本cover一张图的颜色。因为大部分图片的色彩空间是一定的，所以你将图像拉伸shuffle再sample不能说100%，但是99.9%能cover大部分颜色。取在边缘是因为我们其实matting只是分割感兴趣的物体，所以只用在意interested object的边缘信息就能很好的将其“抠”出来。  
  
第二个问题是通过设计一个目标函数，使得前景和背景的样本是源于一个能较好对其进行分割的分布中。

## 2. 再看介绍

又是辣个公式  
$$I_z = \alpha F_z + (1 - \alpha)B_z$$  
  
其中关于得到最佳前景-后景（背景）匹配的$(F,B)$  
$$ \alpha_z = \frac{(I_z - B)\cdot(F-B)}{\left\| F - B\right\|^2}$$

## 3.本文方法

介绍里面关于之前算法的分析太冗余了跳过，这么多年了早期有名的有研究价值的算法都已经沉淀下来，所以再去仔细研究这部分没什么意义。

### 3.1 Gathering comprehensive sample set

这部分我理解为“收集有意义的/可理解的样本集”，实际上就是收集能cover所有前景-背景颜色空间的样本集。这部分一定是与interested object有关系的。该部分做法具体分为两步:  
1. 首先，收集什么样本（即哪个像素点被选为样本集），是根据到已知的前景背景中像素点的距离变化的。  
    *这很好理解，离该前景/背景部分越近的像素点，是其一部分的概率越高。然而，之前的算法采集样本仅考虑边界部分，而本文既考虑了边界，又从区域内部选取样本。*
2. 其次，保证了图像的颜色空间能被采样的范围涵盖（cover）